# Use Model Context Protocol (MCP) as tools with Strands Agent

## Overview
The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Strands AI SDK integrates with MCP to extend agent capabilities through external tools and services.

MCP enables communication between agents and MCP servers that provide additional tools. The Strands Agent SDK includes built-in support for connecting to MCP servers and using their tools.

In this example we will show you how to use MCP tools on your Strands Agent. We will use the [AWS Documentation MCP server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/) which provides tools to access AWS documentation, search for content, and get recommendations. This MCP server has 3 main features:

- **Read Documentation**: Fetch and convert AWS documentation pages to markdown format
- **Search Documentation**: Search AWS documentation using the official search API
- **Recommendations**: Get content recommendations for AWS documentation pages



## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |MCP Tools                                          |
|Agent Structure     |Single agent architecture                          |

</div>

## Architecture

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

## Key Features
* **Single agent architecture**: this example creates a single agent that interacts with MCP tools
* **MCP tools**: Integration of MCP tools with your agent

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.7 enabled on Amazon Bedrock

Let's now install the requirement packages for our Strands Agent agent

In [13]:
# installing pre-requisites
!pip install -r requirements.txt

### Importing dependency packages

Now let's import the dependency packages

In [14]:
import threading
import time
from datetime import timedelta

from mcp import StdioServerParameters, stdio_client
from mcp.client.streamable_http import streamablehttp_client
from mcp.server import FastMCP
from strands import Agent
from strands.tools.mcp import MCPClient

### Connect to MCP server using stdio transport

[Transports](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports) in MCP provide the foundations for communication between clients and servers. It handles the underlying mechanics of how messages are sent and received. At the moment there are three standards transport implementations built-in in MCP:

- **Standard Input/Output (stdio)**: enables communication through standard input and output streams. It is particularly useful for local integrations and command-line tools
- **Streamable HTTP**: this replaces the HTTP+SSE transport from previous protocol version. In the Streamable HTTP transport, the server operates as an independent process that can handle multiple client connections. This transport uses HTTP POST and GET requests. Server can optionally make use of Server-Sent Events (SSE) to stream multiple server messages. This permits basic MCP servers, as well as more feature-rich servers supporting streaming and server-to-client notifications and requests.
- **SSE**: legacy transport for HTTP-based MCP servers that use Server-Sent Events transport  

Overall, you should use stdio for building command-line tools, implementing local integrations and working with shell scripts. You should use Streamable HTTP transports when you need a flexible and efficient way for AI agents to communicate with tools and services, especially when dealing with stateless communication or when minimizing resource usage is crucial.

You can also use **custom transports** implementation for your specific needs. 


Let's now connect to the MCP server using stdio transport. First of all, we will use the class `MCPClient` to connect to the [AWS Documentation MCP Server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/). This server provides tools to access AWS documentation, search for content, and get recommendations.

In [15]:
# Connect to an MCP server using stdio transport
stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `stdio_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [16]:
# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent("What is Amazon Bedrock pricing model. Be concise.")

I'll help you find information about Amazon Bedrock's pricing model. Let me search for the official AWS documentation on this topic.
Tool #1: search_documentation
Now let me get the detailed pricing information from the most relevant link:
Tool #2: read_documentation
Let me also get information about Provisioned Throughput since it's mentioned as part of the pricing model:
Tool #3: read_documentation
Based on the AWS documentation, here's a concise explanation of Amazon Bedrock's pricing model:

## Amazon Bedrock Pricing Model

Amazon Bedrock uses a pay-as-you-go pricing model with two main components:

1. **On-demand Usage**: 
   - Pay based on the volume of input and output tokens processed
   - Different foundation models have different token rates
   - No upfront costs or commitments required

2. **Provisioned Throughput**:
   - Optional fixed-cost option for higher throughput
   - Billed hourly based on:
     - Model selected
     - Number of Model Units (MUs) purchased
     - Com

### Connect to MCP server using Streamable HTTP

Let's now connect to the MCP server using Streamable HTTP transport. First let's start a simple MCP server using Streamable HTTP transport. 

For this example we will create our own MCP server. The architecture will look as following

<div style="text-align:center">
    <img src="images/architecture_2.png" width="65%" />
</div>

In [28]:
# Create an MCP server
mcp = FastMCP("Calculator Server")

# Define a tool


@mcp.tool(description="Calculator tool which performs calculations")
def calculator(x: int, y: int) -> int:
    return x + y


@mcp.tool(description="This is a long running tool")
def long_running_tool(name: str) -> str:
    time.sleep(25)
    return f"Hello {name}"


def main():
    mcp.run(transport="streamable-http", mount_path="mcp")

Let's now start a thread with the `streamable-http` server

In [29]:

thread = threading.Thread(target=main)
thread.start()

INFO:     Started server process [5686]
INFO:     Waiting for application startup.


[12/18/25 17:51:55] INFO     StreamableHTTP session manager started                  ]8;id=357882;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=841592;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#110\110]8;;\

INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('127.0.0.1', 8000): [errno 98] address already in use
INFO:     Waiting for application shutdown.


                    INFO     StreamableHTTP session manager shutting down            ]8;id=507148;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=252287;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#114\114]8;;\

INFO:     Application shutdown complete.


#### Integrating Streamable HTTP client with Agent

Now let's use `streamablehttp_client` integrate this server with a simple agent. 

In [30]:
def create_streamable_http_transport():
    return streamablehttp_client("http://localhost:8000/mcp")


streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `streamable_http_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [31]:
with streamable_http_mcp_client:
    tools = streamable_http_mcp_client.list_tools_sync()

    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = str(agent("What is 2 + 2?"))

[12/18/25 17:52:12] INFO     Created new transport with session ID:                  ]8;id=952805;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=213575;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             176e2e66416b4fe9bdd62a38b5da1a99                                                      

INFO:     127.0.0.1:58466 - "POST /mcp HTTP/1.1" 200 OK


                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=137033;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=954949;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 176e2e66416b4fe9bdd62a38b5da1a99           ]8;id=543454;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=295306;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=26866;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=357244;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#146\146]8;;\

...

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=248649;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=715071;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

The result of 2 + 2 is 4.

[12/18/25 17:52:15] INFO     Terminating session: 176e2e66416b4fe9bdd62a38b5da1a99           ]8;id=410151;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=587601;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py#646\646]8;;\

INFO:     127.0.0.1:48712 - "DELETE /mcp HTTP/1.1" 200 OK


                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=872389;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=737491;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

### Direct Tool Invocation

While tools are typically invoked by the agent based on user requests, you can also call MCP tools directly. This can be useful for workflow scenarios where you orchestrate multiple tools together.

In [9]:
query = {"x": 10, "y": 20}

with streamable_http_mcp_client:
    # direct tool invocation
    result = streamable_http_mcp_client.call_tool_sync(
        tool_use_id="tool-123", name="calculator", arguments=query
    )

    
    # Process the result
    print(f"Calculation result: {result['content'][0]['text']}")

[12/18/25 15:58:38] INFO     Created new transport with session ID:                  ]8;id=782060;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=575798;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             7b6ce7603a0e4635a06446426ab1f597                                                      

INFO:     127.0.0.1:60696 - "POST /mcp HTTP/1.1" 200 OK


                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=19099;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=778739;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 7b6ce7603a0e4635a06446426ab1f597           ]8;id=932175;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=782923;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=959761;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=913655;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#146\146]8;;\

...

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=134374;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=128882;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

Calculation result: 30


                    INFO     Terminating session: 7b6ce7603a0e4635a06446426ab1f597           ]8;id=152334;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=372931;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py#646\646]8;;\

INFO:     127.0.0.1:60760 - "DELETE /mcp HTTP/1.1" 200 OK


                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=151019;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=796483;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

You can optionally also provide `read_timeout_seconds` while calling an MCP server tool to avoid it running for too long

In [32]:
with streamable_http_mcp_client:
    try:
        result = streamable_http_mcp_client.call_tool_sync(
            tool_use_id="tool-123",
            name="long_running_tool",
            arguments={"name": "Amazon"},
            read_timeout_seconds=timedelta(seconds=30),
        )

        if result["status"] == "error":
            print(f"Tool execution failed: {result['content'][0]['text']}")
        else:
            print(f"Tool execution succeeded: {result['content'][0]['text']}")
    except Exception as e:
        print(f"Tool call timed out or failed: {str(e)}")

[12/18/25 17:52:42] INFO     Created new transport with session ID:                  ]8;id=241737;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=710617;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             843cdcf54f2f415cab6569441175a9d2                                                      

INFO:     127.0.0.1:47736 - "POST /mcp HTTP/1.1" 200 OK


                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=124635;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=447540;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 843cdcf54f2f415cab6569441175a9d2           ]8;id=201369;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=64821;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=28256;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=110835;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#146\146]8;;\

...

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=882581;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=774830;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

Tool execution succeeded: Hello Amazon


                    INFO     Terminating session: 843cdcf54f2f415cab6569441175a9d2           ]8;id=262527;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=641273;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py#646\646]8;;\

INFO:     127.0.0.1:49946 - "DELETE /mcp HTTP/1.1" 200 OK


                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=687001;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=538991;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

### Interacting with multiple MCP servers

With Strands Agents you can also interact with multiple MCP servers using the same agent and configure tools setups such as the max number of tools that can be used in parallel (`max_parallel_tools`). Let's create a new agent to showcase this configuration:

<div style="text-align:center">
    <img src="images/architecture_3.png" width="85%" />
</div>

In this agent, we will again use the AWS Documentation MCP server and we will also use the [AWS CDK MCP Server](https://awslabs.github.io/mcp/servers/cdk-mcp-server/) which helps with AWS Cloud Development Kit (CDK) best practices, infrastructure as code patterns and security compliance with CDK Nag.

First let's connect to the two MCP servers using the stdio transport

In [33]:
# Connect to an MCP server using stdio transport
aws_docs_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# Connect to an MCP server using stdio transport
cdk_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(command="uvx", args=["awslabs.cdk-mcp-server@latest"])
    )
)

#### Create Agent with MCP servers

Next we will create the agent with the tools from both MCP servers

In [ ]:
# Create an agent with MCP tools
with aws_docs_mcp_client, cdk_mcp_client:
    # Get the tools from the MCP server
    tools = aws_docs_mcp_client.list_tools_sync() + cdk_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent(
        "What is Amazon Bedrock pricing model. Be concise. Also what are the best practices related to CDK?"
    )

I'll answer both parts of your question concisely - the Amazon Bedrock pricing model and CDK best practices.

## Amazon Bedrock Pricing Model

Let me search for the most accurate pricing information for Amazon Bedrock:
Tool #1: search_documentation
Now let me get the details about Amazon Bedrock pricing:
Tool #2: read_documentation
Let me also check for CDK best practices:
Tool #3: CDKGeneralGuidance
Now I'll answer your questions concisely:

## Amazon Bedrock Pricing Model

Amazon Bedrock uses a pay-as-you-go pricing model based on:

1. **Token-based pricing**: You pay for the volume of both input tokens and output tokens processed
2. **Model-specific rates**: Different foundation models have different pricing tiers
3. **Provisioned Throughput option**: Optional purchase for increased model invocation capacity at potentially lower rates

You only pay for the specific resources you use, with no upfront costs. For detailed pricing information, you can visit the Amazon Bedrock pricing pa

### Congratulations!

In this notebook you learned how to connect with MCP servers using Strands Agent and two MCP transport protocols: stdio and Streamable HTTP. You also learned how to connect multiple MCP servers to the same agent. Next, let's see how to use different models with your agent